In [1]:
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [3]:
# 산업군별 코드 삽입
# 1.정부/공공기관/공기업(1001), 2.은행/금융업(900), 3.솔루션/SI/ERP/CRM(707), 4.네트워크/통신/모바일(704)
# 5.미디어/디자인(800), 6.세무/회계(1004), 7.전기/전자/제어(201), 8.서비스업(100)
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [4]:
# [미디어/디자인] 산업군의 기업 리뷰 스크래핑
reviews_5 = dict(ind_reviews(800))

In [5]:
reviews_5

{'소기업. 중소도 아닌 소기업. 가족회사. 방송국이라 하기엔 너무 작은 개인회사': '36',
 '별점 1도 아까운데..잡플래닛이 사람 여럿살리는 역할하는듯. 이거 보시고 가지마세요 여기. 최악. 고용노동부에 신고 넣었는데 내 앞으로 신고자가 9명이라 그랬음': '36',
 '즐겁고 쾌할한 회사 분위기에서 온라인마케팅 관련 전반에 대해 배울 수 있는 수 있는 회사': '36',
 '광고 대행사라는 타이틀을 걸고 있지만, 실질적으로는 광고송출 대행사에 맞는 평가. AE 포지션에서 타 광고 대행사보다 할 수 있는 일이 거의 없다고 봐도 무방.': '36',
 '개발자가 대우받으며 일할 수 없습니다요': '36',
 '조직원의 역량을 높여주는 교육 기회를 제공해준다.': '36',
 '디자인보다는 서류쓰기나 자료조사가 더 많은회사': '36',
 '연봉이 오르지않는다. 별도의 수당이 없다': '36',
 '면접때 알 수 없는 질문있었지만 기본적인 질문 했어요.': '36',
 '빛좋은 개살구.. 되살아날 수 있을까 싶은 대표의 프라이드만 높은 회사': '36',
 '가족같은 회사로 화목해 보일 수 있고 도가 넘어 보일 수 있다 그래도 디자인 일은 꾸준히 들어와서 신입은 디자인을 많이 해볼 수 있다 그리고 회사전체가 고인물이다': '54',
 '일에만 집중하고 개인주의적인 성격이라면 좋은 직장\r인간관계 신경쓰는 사람이면  힘든 직장\r     .': '54',
 '디자인 툴을 사용할 줄 안다면 어려움은 없는 회사.하지만 영업사원 및 클라이언트를 직접 상대해야해서 커뮤니케이션으로 인한 스트레스는 있을 수 있음': '54',
 '다양한 종합 편성프로그램을 경험할 수 있어서 여러가지 노하우와 경험은 쌓을 수 있음': '54',
 '실제 배우로서 일했던 썰 풀어드립니다.': '54',
 '수직 문화, 짠 내 나는 사장님,포폴이 쓸데가 없다작은 회사의 특징. 멋모르는 어린 양들 데려다멀티로 돌리며 모든 예산을 줄이는 곳..?': '54',
 '뭐하는 회사인지 정체성이 불분

In [6]:
len(reviews_5)

300

In [7]:
df = pd.DataFrame(reviews_5, index = [1])
df = df.transpose()
df.columns = ['score']
df['review'] = df.index
df.insert(0, 'industry', 'media_design')
df.reset_index(drop=True)

,industry,score,review
0,media_design,36,소기업. 중소도 아닌 소기업. 가족회사. 방송국이라 하기엔 너무 작은 개인회사
1,media_design,36,별점 1도 아까운데..잡플래닛이 사람 여럿살리는 역할하는듯. 이거 보시고 가지마세요...
2,media_design,36,즐겁고 쾌할한 회사 분위기에서 온라인마케팅 관련 전반에 대해 배울 수 있는 수 있는 회사
3,media_design,36,"광고 대행사라는 타이틀을 걸고 있지만, 실질적으로는 광고송출 대행사에 맞는 평가. ..."
4,media_design,36,개발자가 대우받으며 일할 수 없습니다요
...,...,...,...
295,media_design,64,인천에 이런 대우 해주는 회사 없어요너무 편하게 회사 다니시려는 분께는 어디든 힘들...
296,media_design,64,사내 분위기가 나쁘지 않다 평타는 치는 느낌?법에 민감해서 위법은 절대 안할 기업
297,media_design,64,홍보에이전시지만 칼퇴가능하고 연차 사용이 자유로운 편\r그치만 소기업이라서 불편한 ...
298,media_design,64,"자유로운 연차사용, 개인의 생각이나 원하는 것을 자유롭게 도전해볼 수 있는 곳"


In [8]:
# [세무/회계] 산업군의 기업 리뷰 스크래핑
reviews_6 = dict(ind_reviews(1004))

In [9]:
reviews_6

{'인수인계 없고 업무체계 없고 정치색 강요와 성적인 요소로 불쾌감 조성 및 각종 비하를 해주는 특별한 곳!': '36',
 '개인주의자 성향이 강한 회사여서 저하고 잘 맞았습니다.': '36',
 '신입직원이 일하기 좋은 사무실이라고 생각합니다': '36',
 '근로자 시간을 갈아서 유지하는 회사자신의 인생을 살고자 하는 사람들에게는 최악의 회사': '36',
 '내가 하는일에 아무도 터치하지 않아서 편히 일하기 좋다.': '36',
 '크게 기대할 것도 실망할 것도 없는 정도임\r적당히 경력 쌓을 정도임.': '36',
 '편하게 일하는 곳 시키는 일만 하면 되는 곳': '36',
 '배우고자 한다면 많은 것을 배울수 있으나, 그런 의지가 없다면 시간만 흘려보낼 것 같음. 난이도가 살짝 있는 업무를 하기위해서는 개인적인 공부가 필수. 그래야 물어봤을때 알려주시는걸 빨리 흡수할 수 있음.': '36',
 '조직문화가 심각한 곳 야근이 잦은 곳 사장님이 큰 소리를 내도 직원들이 아무말도 못하는 곳': '36',
 '적은 월급에 비해 많은 노동력 가성비가 딸림다양한 거래처로 배울점은 많으나 체계적이지 못한 교육방식': '36',
 '다양한 일을 배울 수 있고 자유로운 분위기인 편임.': '94',
 '기장팀이라 팀마다 다르겠지만 개인주의라면 자유롭고 편합니다': '90',
 '다양한 경험을 할 수 있고 문화 또하 좋았습니다.': '86',
 '작은곳이지만 마음만은 편한 회사였다 \r퇴사했어도 가끔 연락하고 좋았다': '86',
 '사람들이 너무 좋았고 좋은 상사를 만나서 많이 배울 수 있었음': '86',
 '결산, 조정 등 업무 배우기에 좋은 환경임': '86',
 '다양한 업무를 모두 경험할 수 있음 여의도에 위치해있어 출퇴근이 용이하고 식사나 커피 모두 다양하게 선택 가능함': '80',
 '지점이 여러개인데 지점끼리 서로 교류가 없음 지점마다 다르지만 일단 일이 많고 야근이 많음': '80',
 '전문지식을 배우며 업무를 빠르고 체계적으로 배울 수 있는 